In [ ]:
from module.jupyter_util import *
%matplotlib notebook

# fetch_network_info() fetches the topology from API service for all topologies in the topology MySQL table
# network_info = {id: {"topology": {<topology from E2E>}}}  where id is from the topology SQL table
network_info = fetch_network_info()
# first topology in the list
t_id = min(network_info.keys())
t = network_info[t_id]["topology"]
# use current time, or specific time in past
end_time = int(time.time()) # int(datetime.strptime("2019-03-28 17:27:00 -0700", "%Y-%m-%d %H:%M:%S %z").timestamp())
# we store stats at 1s, 30s, and 900s intervals
read_interval = 30
# window in seconds in past
window = 300

nts = NumpyTimeSeries(end_time - window, end_time, read_interval, network_info)

# output format of read_stats() is numpy array
# it can be StatType.NODE or StatType.LINK
# missing stats are shown as np.nan

mcs = nts.read_stats("staPkt.mcs", StatType.LINK)
# mcs[ti][li, di, :] 
# where ti is the topology index, li is the link index,  di is the direction (0,1), and : is the time axis
mlu = nts.read_stats("staPkt.mgmtLinkUp", StatType.LINK)
# calculate and print table of average of these stats, if mcs < 9
mean_mcs = np.round(np.nanmean(mcs[0], axis=nts.TIME_AXIS), 1)
mean_mlu = np.round(np.nanmean(mlu[0], axis=nts.TIME_AXIS), 1)
tabulate_links(["a_node_mac", "z_node_mac"], [mean_mcs, mean_mlu], (mean_mcs < 10).any(axis=nts.DIR_AXIS), t)

# similarly create table for node stat
sync_mode_gps = nts.read_stats("tsf.syncModeGps", StatType.NODE)[0]
num_missing = np.isnan(sync_mode_gps).sum(axis=nts.TIME_AXIS)
mean_smg = np.round(np.nanmean(sync_mode_gps, axis=nts.TIME_AXIS), 1)
tabulate_nodes(["name", "mac_addr", "node_type"], [mean_smg, num_missing], np.logical_and(mean_smg == 1, num_missing < 5) , t)

# print unique values of mcs
show_uniques(mcs[0])

# plot mcs in time
plt.plot(mcs[0][0, 0, :], '.-')
plt.grid()